# Predict Water Amount Needed

## Get and Prepare Weather Data from DataBase

In [80]:
import pymysql.cursors
import pandas as pd
import numpy as np

pd.options.display.max_rows = 200
#pd.set_option('display.float_format', lambda x: '%.20f' % x) #Display as Float
pd.set_option('display.float_format', lambda x: '{:,}'.format(x)) #Display as Scientific


connection = pymysql.connect(host = "mikmak.cc", user="sensor", passwd="Gaffe2017", db="weatherDW")
query = ('SELECT * FROM log_v_last24Hours WHERE S_Text <> "None"')

with connection.cursor() as cursor:
    cursor.execute(query)
connection.commit()
e_Log = cursor.fetchall()
connection.close()

e_Log = (np.array(e_Log))
e_Log

144

array([['Wind', '10.4', datetime.datetime(2017, 4, 15, 19, 51, 41)],
       ['Temperature', '12.0', datetime.datetime(2017, 4, 15, 19, 51, 41)],
       ['Rain', '0.0', datetime.datetime(2017, 4, 15, 19, 51, 41)],
       ['Pressure', '960.8', datetime.datetime(2017, 4, 15, 19, 51, 41)],
       ['Sunshine', '0', datetime.datetime(2017, 4, 15, 19, 51, 41)],
       ['Humidity', '62', datetime.datetime(2017, 4, 15, 19, 51, 41)],
       ['Pressure', '960.6', datetime.datetime(2017, 4, 15, 18, 51, 40)],
       ['Sunshine', '0', datetime.datetime(2017, 4, 15, 18, 51, 40)],
       ['Humidity', '77', datetime.datetime(2017, 4, 15, 18, 51, 40)],
       ['Wind', '7.6', datetime.datetime(2017, 4, 15, 18, 51, 40)],
       ['Temperature', '11.4', datetime.datetime(2017, 4, 15, 18, 51, 40)],
       ['Rain', '0.0', datetime.datetime(2017, 4, 15, 18, 51, 40)],
       ['Wind', '5.8', datetime.datetime(2017, 4, 15, 17, 51, 40)],
       ['Temperature', '11.8', datetime.datetime(2017, 4, 15, 17, 51, 40)],
 

In [81]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

t_Log = e_Log

t_Log = pd.DataFrame(data = t_Log[1:,:],
                     index = t_Log[1:,2],
                     columns =["ID","Value","TimeStamp"] )

t_Log = t_Log[t_Log != '-'] #DataSet contains some missing values, remove them

t_Log = t_Log.pivot(index='TimeStamp', columns='ID', values='Value')
t_Log = t_Log.sort_index(ascending = True) #It should be ascending, for rolling calculation later
t_Log = t_Log.dropna(how = "any")
t_Log = t_Log.astype(float)

t_Log.head()


ID,Humidity,Pressure,Rain,Sunshine,Temperature,Wind
TimeStamp,,,,,,
2017-04-14 20:51:33,69.0,959.5,0.0,0.0,12.2,1.8
2017-04-14 21:51:33,52.0,960.2,0.0,0.0,12.1,6.5
2017-04-14 22:51:34,52.0,960.3,0.0,0.0,11.8,6.8
2017-04-14 23:51:34,52.0,960.4,0.0,0.0,11.7,10.8
2017-04-15 00:51:34,57.0,960.3,0.0,0.0,11.1,8.3


## Transform Data into Input Vector

### Order Data and Calculate Means and Sums

In [82]:
X = pd.DataFrame()

lastRow = len(t_Log.axes[0]) - 1

X = X.assign(tre200b0=t_Log.Temperature[[lastRow]])
X = X.assign(ure200b0=t_Log.Humidity[[lastRow]])
X = X.assign(rre200b0=t_Log.Rain[[lastRow]])
X = X.assign(sre000b0=t_Log.Sunshine[[lastRow]])
X = X.assign(fu3010b0=t_Log.Wind[[lastRow]])
X = X.assign(prestab0=t_Log.Pressure[[lastRow]])
#"tre200b0","ure200b0","rre150b0","sre000b0","fu3010b0","prestab0"

t_LogRolling = t_Log.rolling(len(t_Log.axes[0]))

X = X.assign(tre200b0_mean=t_LogRolling.Temperature.mean())
X = X.assign(ure200b0_mean=t_LogRolling.Humidity.mean())
X = X.assign(rre200b0_sum=t_LogRolling.Rain.sum())
X = X.assign(sre000b0_sum=t_LogRolling.Sunshine.sum())
X = X.assign(fu3010b0_mean=t_LogRolling.Wind.mean())
X = X.assign(prestab0_mean=t_LogRolling.Pressure.mean())

X.transpose()

TimeStamp,2017-04-15 18:51:40
tre200b0,11.4
ure200b0,77.0
rre200b0,0.0
sre000b0,0.0
fu3010b0,7.6
prestab0,960.6
tre200b0_mean,11.88695652173913
ure200b0_mean,59.869565217391305
rre200b0_sum,0.0
sre000b0_sum,3.0


### Poly Features etc.

In [83]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.preprocessing import PolynomialFeatures

polyDegree = 5
poly = PolynomialFeatures(degree=polyDegree)
X = poly.fit_transform(X).astype(int)

#poly.n_output_features_ 
#pd.DataFrame(X).transpose()

## Classify!
Use Logistic Regression Modell to decide if 
- no water at all or 
- a lot of water or 
- normal amount of water
is needed

### Load Model

In [84]:
from sklearn.externals import joblib
model = joblib.load('plant.Watering.data/logreg_5deg.pkl')

### Do Calculation and Show Result
- 0 - means a lot of water
- 1 - is medium
- 2 - device should stay off

In [85]:
result = model.predict(X)[0]

from sklearn import preprocessing
le = joblib.load('plant.Watering.data/leModel.pkl')
le.inverse_transform(result)


'Med'

This is very good!

In [86]:
#pd.DataFrame(model.coef_.transpose())

In [87]:
if

SyntaxError: invalid syntax (<ipython-input-87-f78ffcb5c1e0>, line 1)